In [1]:
# Import modules
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
#load data and take out id columns
new_data = pd.read_csv('data/final_data.csv')

In [3]:
#numeric columns
num =['time_in_hospital','num_lab_procedures','num_procedures','num_medications','number_outpatient','number_emergency',
'number_inpatient','number_diagnoses']

In [5]:
#normalize new_data numeric columns
from sklearn.preprocessing import Normalizer
transformer = Normalizer().fit(new_data[num])
new_data[num] = transformer.transform(new_data[num])
new_data[num] = pd.DataFrame(new_data[num], columns= new_data[num].columns)

In [6]:
#Dummify
new_data = pd.get_dummies(new_data, drop_first = True)
new_data.shape

(84432, 151)

In [ ]:
# #Up-sample Minority Class
# from sklearn.utils import resample
# majority = new_data[new_data.target==0]
# minority = new_data[new_data.target==1]
# minority_upsampled = resample(minority,replace=True,n_samples=76847,random_state=123)
# new_data = pd.concat([majority, minority_upsampled])
# new_data.target.value_counts()

In [7]:
#Spliting to training
target = new_data.target
X_train, X_test, y_train, y_test = train_test_split(new_data, target, test_size=0.25, stratify = target)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(63324, 151) (63324,)
(21108, 151) (21108,)


In [ ]:
# #Spliting to testing datasets
# target = new_data.target
# X_train1, X_test1, y_train1, y_test1 = train_test_split(new_data, target, test_size=0.25, stratify = target)
# print(X_train1.shape, y_train1.shape)
# print(X_test1.shape, y_test1.shape)

In [8]:
#drop target from training
X_train = X_train.drop('target', axis=1)
X_test = X_test.drop('target', axis=1)

In [55]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.linear_model import LogisticRegression
# grid={"C":np.logspace(-3,3,7), "penalty":["l1"]}
# logreg=LogisticRegression(class_weight={0:1, 1:8})
# logreg_cv=GridSearchCV(logreg,grid,cv=3)
# logreg_cv.fit(X_train,y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight={0: 1, 1: 8}, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]), 'penalty': ['l1']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [56]:
# logreg_cv.best_params_

{'C': 0.1, 'penalty': 'l1'}

In [60]:
## Fit the model with training
decision_logit = LogisticRegression(C=0.1, penalty= 'l1', random_state=0,class_weight= {0:1, 1:8})
decision_logit.fit(X_train, y_train)

## The score (accuracy for classification problems):
decision_logit.score(X_train, y_train)

0.648063925210031

In [61]:
#AUC test  
from sklearn.metrics import roc_auc_score
upsample_AUC_train =roc_auc_score(y_test, decision_logit.predict(X_test))
upsample_AUC_train

0.6137530619768308

In [62]:
#confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, decision_logit.predict(X_test))


array([[12448,  6256],
       [ 1053,  1351]])